In [5]:
# Chicago API
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
def getBusStop(df, spectag):
    tagdf = df.find_all(spectag)
    returndf = []
    for tag in tagdf:
        temp = tag.text.strip() 
        returndf.append(temp) 
    returndf = pd.Series(returndf)
    return returndf

routesurl = "http://www.ctabustracker.com/bustime/api/v3/getroutes?key=kUXKRKMhBsjB3a32q5UcLZ4YX"
routes = requests.get(url=routesurl)
routesdata = BeautifulSoup(routes.text,"xml")
routetag = routesdata.find_all("rt")
routenumbers = []
for tag in routetag: 
    temp = tag.text.strip() 
    routenumbers.append(temp) 
routenumbers = pd.Series(routenumbers) 
templateurl = "https://www.ctabustracker.com/bustime/api/v3/getdirections?key=kUXKRKMhBsjB3a32q5UcLZ4YX&rt="
templateurl2 = "https://www.ctabustracker.com/bustime/api/v3/getstops?key=kUXKRKMhBsjB3a32q5UcLZ4YX&rt="
templateurl3 = "&dir="
stopsloc = pd.DataFrame(columns=["StopId","Lat","Lon"])
for i in routenumbers:
    directionurl = "".join([templateurl, i])
    direction = requests.get(url=directionurl)
    directiondata = BeautifulSoup(direction.text,"xml")
    directiontag = directiondata.find_all("id")
    dir = []
    for tag in directiontag: 
        temp = tag.text.strip() 
        dir.append(temp) 
    dir = pd.Series(dir) 
    stopurl = "".join([templateurl2, i])
    stopurl = "".join([stopurl, templateurl3])
    for w in dir:
        stopurltemp = "".join([stopurl, w])
        stops = requests.get(url=stopurltemp)
        stopsdata = BeautifulSoup(stops.text, "xml")
        StopId = getBusStop(stopsdata, "stpid")
        Lat = getBusStop(stopsdata, "lat")
        Lon = getBusStop(stopsdata, "lon")
        stops_df = pd.DataFrame({'StopId': StopId, "Lat": Lat, "Lon": Lon})
        stopsloc = pd.concat([stopsloc, stops_df], axis =0)




In [11]:
stopsloc = stopsloc.iloc[:,[0,2,1]]
stopsloc

,StopId,Lat,Lon
0,1564,41.840606999999,-87.623207000001
1,12713,41.879288000001,-87.639435000001
2,17046,41.879457,-87.631083
3,1108,41.879177000001,-87.636101000002
4,4732,41.879491,-87.628093000001
...,...,...,...
81,14258,42.030722,-87.685911999999
82,18322,42.023165999999,-87.685016000001
83,14254,42.026477,-87.685354999999
84,14256,42.028933,-87.685657000001


In [ ]:
import os
os.getcwd()
ChicagoStops = stopsloc
DCStops = pd.read_csv("Metro_Bus_Stops.csv")
DCStops.columns=["StopId","Lon","Lat"]
LAStops = pd.read_csv("stops.txt")
LAStops = LAStops[["stop_id","stop_lat","stop_lon"]]
LAStops.columns={"StopId","Lon","Lat"}
ChicagoStops = ChicagoStops[["StopId","Lon","Lat"]]
ChicagoStops["City"] = "Chicago"
DCStops["City"] = "DC"
LAStops["City"] = "Los Angeles"
BusStops = pd.concat([ChicagoStops,LAStops,DCStops],axis=0)


In [15]:
BusStops

,StopId,Lon,Lat,City
0,1564,-87.623207000001,41.840606999999,Chicago
1,12713,-87.639435000001,41.879288000001,Chicago
2,17046,-87.631083,41.879457,Chicago
3,1108,-87.636101000002,41.879177000001,Chicago
4,4732,-87.628093000001,41.879491,Chicago
...,...,...,...,...
10358,NaN,-77.217035,38.825841,DC
10359,NaN,-76.889212,38.887514,DC
10360,NaN,-77.204532,38.831705,DC
10361,NaN,-77.193622,38.830114,DC


In [22]:
DCRail = pd.read_csv("Metro_Stations_in_DC.csv")
DCRail.columns=["StationId","Lon","Lat"]
LARail = pd.read_csv("la_stops.txt")
LARail = LARail[["stop_id","stop_lat","stop_lon"]]
LARail.columns={"StationId","Lon","Lat"}
ChicagoRail = pd.read_csv("CTA_RailStations.csv")
ChicagoRail["City"] = "Chicago"
DCRail["City"] = "DC"
LARail["City"] = "Los Angeles"
RailStations = pd.concat([ChicagoRail,LARail,DCRail],axis=0)

In [23]:
RailStations

,StationId,Lon,Lat,City
0,580.0,-87.756698,41.851781,Chicago
1,1680.0,-87.747227,42.026252,Chicago
2,150.0,-87.724333,41.853865,Chicago
3,1020.0,-87.708547,41.929736,Chicago
4,550.0,-87.729183,41.952948,Chicago
...,...,...,...,...
35,36.0,-76.985498,38.881271,DC
36,37.0,-77.005088,38.876489,DC
37,38.0,-77.017507,38.87647,DC
38,39.0,-76.995373,38.862971,DC
